In [141]:
import torch
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split

base_path = 'IR-Newspapers-files\\IR-files'

labels_list = {'A-J' : 0, 'BBC' : 1, 'J-P' : 2, 'NY-T' : 3}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(str(device))

cuda


In [142]:
import pandas as pd
import os


def apply_to_all_matrices(classifier):
    for dir in os.listdir(base_path):
        if dir == 'TF-IDF':
            clean_matrices = []
            lemma_matrices = []
            for file in sorted(os.listdir(f'{base_path}\\{dir}')):
                if 'clean' in file:
                    clean_matrices.append(pd.read_excel(f'{base_path}\\{dir}\\{file}').drop(columns=['DocumentIndex']))
                elif 'lemma' in file:
                    lemma_matrices.append(pd.read_excel(f'{base_path}\\{dir}\\{file}').drop(columns=['DocumentIndex']))


            shapes = [i.shape[0] for i in clean_matrices]

            y = []
            for i, j in enumerate(shapes):
                y = y + [i] * j
            y = np.array(y)


            clean_df = pd.concat(clean_matrices, ignore_index=True)
            lemma_df = pd.concat(lemma_matrices, ignore_index=True)



            print(f'{"-"*20}TF-IDF clean{"-"*20}')
            classifier(clean_df.to_numpy(dtype=np.float32), y,'TF-IDF clean')
            print(f'{"-"*20}TF-IDF lemma{"-"*20}')
            classifier(lemma_df.to_numpy(dtype=np.float32), y, 'TF-IDF lemma')

        else:
            for matrix in os.listdir(f'{base_path}\\{dir}'):
                df = pd.read_csv(f'{base_path}\\{dir}\\{matrix}')

                y = df['Sheet'].map(labels_list)

                df.drop(['RowIndex', 'Sheet'], axis=1, inplace=True)


                print(f'{"-"*20}{matrix}{"-"*20}')
                classifier(df.to_numpy(dtype=np.float32), y.to_numpy(), matrix)

In [143]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
import numpy as np
import os

def train_model(model, train_loader, val_loader, criterion, optimizer, max_epochs=15, patience=3, save_path='best_model.pth'):
    """
    Train the model with early stopping based on accuracy and save the best model.

    Parameters:
    - model: The neural network model.
    - train_loader: DataLoader for the training set.
    - val_loader: DataLoader for the validation set.
    - criterion: Loss function.
    - optimizer: Optimizer.
    - device: Device to run the training on ("cpu" or "cuda").
    - max_epochs: Maximum number of training epochs.
    - patience: Early stopping patience.
    - save_path: Path to save the best model.

    Returns:
    - model: The best trained model.
    """
    best_accuracy = 0.0
    epochs_no_improve = 0

    model.to(device)
    for epoch in range(max_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
        train_loss /= len(train_loader.dataset)

        # Validation phase
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)  # Multi-class prediction
                correct += (predicted == targets).sum().item()
                total += targets.size(0)
        val_accuracy = correct / total

        print(f"Epoch {epoch+1}/{max_epochs} | Train Loss: {train_loss:.4f} | Val Accuracy: {val_accuracy:.4f}")


        # Check early stopping based on accuracy
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            epochs_no_improve = 0
            torch.save(model.state_dict(), save_path)
            print(f"Model saved at epoch {epoch+1} with accuracy: {val_accuracy:.4f}")
        else:
            epochs_no_improve += 1
            print(f"No improvement for {epochs_no_improve} epoch(s)")

        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            break

    # Load the best model
    model.load_state_dict(torch.load(save_path, weights_only=True))
    print(f"Best Validation Accuracy: {best_accuracy:.4f}")
    return model.to(device)


In [144]:
from dataset import CDataset
from ANN import ANN
def train_ANN(X, y, name):
    dataset = CDataset(X, y)

    train_size = int(0.72 * len(dataset))
    val_size = int(0.08 * len(dataset))
    test_size = len(dataset) - train_size - val_size

    ANN_Relu = ANN(X.shape[1], 'relu')
    ANN_Gelu = ANN(X.shape[1], 'gelu')

    train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32)
    test_loader = DataLoader(test_dataset, batch_size=32)

    criterion = nn.CrossEntropyLoss()  # Multi-class classification loss
    optimizer_relu = torch.optim.Adam(ANN_Relu.parameters(), lr=0.001)
    optimizer_gelu = torch.optim.Adam(ANN_Gelu.parameters(), lr=0.001)

    trained_relu_model = train_model(ANN_Relu, train_loader, val_loader, criterion, optimizer_relu, save_path=f'{name}_best_relu_model.pth')
    trained_gelu_model = train_model(ANN_Gelu, train_loader, val_loader, criterion, optimizer_gelu, save_path=f'{name}_best_gelu_model.pth')

     # Evaluate on test set
    trained_relu_model.eval()
    trained_gelu_model.eval()
    correct_relu = 0
    correct_gelu = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs_relu = trained_relu_model(inputs)
            outputs_gelu = trained_gelu_model(inputs)
            _, predicted_relu = torch.max(outputs_relu, 1)
            _, predicted_gelu = torch.max(outputs_gelu, 1)
            correct_relu += (predicted_relu == targets).sum().item()
            correct_gelu += (predicted_gelu == targets).sum().item()
            total += targets.size(0)
    test_accuracy_relu = correct_relu / total
    test_accuracy_gelu = correct_gelu / total
    print(f"Test Relu Accuracy: {test_accuracy_relu:.4f}")
    print(f"Test Gelu Accuracy: {test_accuracy_gelu:.4f}")









In [145]:
apply_to_all_matrices(train_ANN)

--------------------bert_withIDF.csv--------------------
Epoch 1/15 | Train Loss: 0.9374 | Val Accuracy: 0.8396
Model saved at epoch 1 with accuracy: 0.8396
Epoch 2/15 | Train Loss: 0.3139 | Val Accuracy: 0.9465
Model saved at epoch 2 with accuracy: 0.9465
Epoch 3/15 | Train Loss: 0.1686 | Val Accuracy: 0.9679
Model saved at epoch 3 with accuracy: 0.9679
Epoch 4/15 | Train Loss: 0.1105 | Val Accuracy: 0.9733
Model saved at epoch 4 with accuracy: 0.9733
Epoch 5/15 | Train Loss: 0.0741 | Val Accuracy: 0.9412
No improvement for 1 epoch(s)
Epoch 6/15 | Train Loss: 0.0681 | Val Accuracy: 0.9786
Model saved at epoch 6 with accuracy: 0.9786
Epoch 7/15 | Train Loss: 0.0402 | Val Accuracy: 0.9679
No improvement for 1 epoch(s)
Epoch 8/15 | Train Loss: 0.0303 | Val Accuracy: 0.9786
No improvement for 2 epoch(s)
Epoch 9/15 | Train Loss: 0.0282 | Val Accuracy: 0.9679
No improvement for 3 epoch(s)
Early stopping triggered.
Best Validation Accuracy: 0.9786
Epoch 1/15 | Train Loss: 0.7916 | Val Accura